In [ ]:
import ipywidgets as widgets

In [ ]:
DEFAULTS = {
    'size': 640,
    'heading': 140,
    'pitch': 10,
    'fov': 50
}

In [ ]:
output = widgets.Output()
lbl_style = {'description_width': '200px'}
layout = widgets.Layout(width='600px')
location = widgets.Text(value='33rd & Loch Raven Baltimore MD',
                        description='Location',
                        layout=layout,
                        style=lbl_style)
size = widgets.IntSlider(value=DEFAULTS['size'],
                         min=100,
                         max=1024,
                         layout=layout,
                         style=lbl_style,
                         description='Image size')
heading = widgets.IntSlider(value=DEFAULTS['heading'],
                            min=0,
                            max=360,
                            layout=layout,
                            style=lbl_style,
                            description='Heading (rotation)')
pitch = widgets.IntSlider(value=DEFAULTS['pitch'],
                          min=0,
                          max=40,
                          layout=layout,
                          style=lbl_style,
                          description='Pitch (tilt)')
fov = widgets.IntSlider(value=DEFAULTS['fov'],
                        min=10,
                        max=120,
                        layout=layout,
                        style=lbl_style,
                        description='Field of view (zoom)')

button = widgets.Button(description='Get image')

display(location, size, heading, pitch, fov, button, output)


def button_click(b):
    with output:
        output.clear_output()
        txt = f'test: location = {location.value}'
        print(txt)
    return txt

sv_img = button.on_click(button_click)
